In [69]:
#Importing All libararies
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, VGG16
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from PIL import Image
import sys
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix

In [70]:
#Saving the directory of correct images in a dataframe along with label
image_id = []
object_type = []
dir1 = '/content/drive/MyDrive/Dockship_Dataset/Garbage_Classification_AI_Challenge/garbage_classification_ai_challenge-dataset/TRAIN/'
for dirname in os.listdir(dir1):
    if dirname!='.DS_Store':
        for filename in os.listdir(dir1+dirname):
              image_id.append(dir1+dirname+'/'+filename)
              object_type.append(dirname)


In [71]:
#total no of images
len(image_id)

2001

In [72]:
df = pd.DataFrame(list(zip(image_id, object_type)),columns = ['ImageID','Type'])

In [73]:
# Shuffling the data frame
df = df.sample(frac=1).reset_index(drop=True)

In [74]:
inp = df['ImageID']
out = df['Type']
inp, out

(0       /content/drive/MyDrive/Dockship_Dataset/Garbag...
 1       /content/drive/MyDrive/Dockship_Dataset/Garbag...
 2       /content/drive/MyDrive/Dockship_Dataset/Garbag...
 3       /content/drive/MyDrive/Dockship_Dataset/Garbag...
 4       /content/drive/MyDrive/Dockship_Dataset/Garbag...
                               ...                        
 1996    /content/drive/MyDrive/Dockship_Dataset/Garbag...
 1997    /content/drive/MyDrive/Dockship_Dataset/Garbag...
 1998    /content/drive/MyDrive/Dockship_Dataset/Garbag...
 1999    /content/drive/MyDrive/Dockship_Dataset/Garbag...
 2000    /content/drive/MyDrive/Dockship_Dataset/Garbag...
 Name: ImageID, Length: 2001, dtype: object, 0         paper
 1         trash
 2       plastic
 3         glass
 4         glass
          ...   
 1996      metal
 1997    plastic
 1998    plastic
 1999      metal
 2000      paper
 Name: Type, Length: 2001, dtype: object)

In [75]:
#splitting the data set   
train_input, test_input, train_output, test_output = train_test_split(inp, out, random_state = 42,test_size = 0.2,stratify = out)      

In [76]:
train_input

878     /content/drive/MyDrive/Dockship_Dataset/Garbag...
516     /content/drive/MyDrive/Dockship_Dataset/Garbag...
458     /content/drive/MyDrive/Dockship_Dataset/Garbag...
846     /content/drive/MyDrive/Dockship_Dataset/Garbag...
596     /content/drive/MyDrive/Dockship_Dataset/Garbag...
                              ...                        
1907    /content/drive/MyDrive/Dockship_Dataset/Garbag...
424     /content/drive/MyDrive/Dockship_Dataset/Garbag...
1025    /content/drive/MyDrive/Dockship_Dataset/Garbag...
1378    /content/drive/MyDrive/Dockship_Dataset/Garbag...
31      /content/drive/MyDrive/Dockship_Dataset/Garbag...
Name: ImageID, Length: 1600, dtype: object

In [77]:
IMG_SIZE = 200
size = (IMG_SIZE,IMG_SIZE)

In [78]:
# Creating an object for imagedatagenerator for generating data in bacthes and augmentation
datagen = ImageDataGenerator(
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest'
)

In [79]:
train = pd.concat([train_input,train_output],axis=1)

In [80]:
# Visualising the class soplit
from collections import Counter 
Counter(object_type)

Counter({'cardboard': 323,
         'glass': 401,
         'metal': 328,
         'paper': 476,
         'plastic': 386,
         'trash': 87})

In [51]:
train_generator = datagen.flow_from_dataframe(
                    train,
                    x_col = "ImageID",
                    y_col = "Type",
                    target_size = size,
                    class_mode = "sparse",
                    batch_size = 64,
                    shuffle = True,
                    seed = 42,
                    interpolation = "nearest"
)

Found 1600 validated image filenames belonging to 6 classes.


In [52]:
test = pd.concat([test_input,test_output],axis=1)

In [53]:
valid_generator = datagen.flow_from_dataframe(
                    test,
                    x_col = "ImageID",
                    y_col = "Type",
                    target_size = size,
                    class_mode = "sparse",
                    batch_size = 64,
                    shuffle = False,
                    seed = 42,
                    interpolation = "nearest"
)

Found 401 validated image filenames belonging to 6 classes.


In [54]:
NUM_CLASSES = 6

In [55]:
def create_model():
    model = models.Sequential()
    model.add(EfficientNetB0(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet'))
    model.add(layers.MaxPool2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation = 'relu'))
    model.add(layers.Dropout(0.20))
    model.add(layers.Dense(NUM_CLASSES, activation = 'softmax'))
    return model

In [24]:
model = create_model()

16711680/16705208 [==============================] - 0s 0us/step


In [25]:
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = Adam(learning_rate = 0.001),
             metrics = ['accuracy'])

In [26]:
# Stop training when the validation loss metric has stopped decreasing for 5 epochs.
early_stopping = EarlyStopping(monitor = 'val_loss',
                               patience = 5,
                               mode = 'min',
                               restore_best_weights = True)

# Save the model with the minimum validation loss
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Dockship_Dataset/Models/best_model.hdf', 
                             monitor = 'val_loss',
                             verbose = 1,
                             mode = 'min', 
                             save_best_only = True)
# reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              min_lr = 0.001,
                              mode = 'min',
                              verbose = 1)

In [49]:
#Trying with 30 epochs
history = model.fit(train_generator,
                    validation_data = valid_generator,
                    epochs = 30,
                    callbacks = [early_stopping, checkpoint, reduce_lr]
                    )

Epoch 1/30
25/25 [==============================] - 28s 1s/step - loss: 0.0652 - accuracy: 0.9794 - val_loss: 0.8019 - val_accuracy: 0.8853

Epoch 00001: val_loss improved from inf to 0.80186, saving model to /content/drive/MyDrive/Dockship_Dataset/Models/best_model.hdf
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Dockship_Dataset/Models/best_model.hdf/assets
Epoch 2/30
25/25 [==============================] - 28s 1s/step - loss: 0.1126 - accuracy: 0.9775 - val_loss: 0.5985 - val_accuracy: 0.8653

Epoch 00002: val_loss improved from 0.80186 to 0.59847, saving model to /content/drive/MyDrive/Dockship_Dataset/Models/best_model.hdf
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Dockship_Dataset/Models/best_model.hdf/assets
Epoch 3/30
25/25 [==============================] - 28s 1s/step - loss: 0.1195 - accuracy: 0.9694 - val_loss: 0.5741 - val_accuracy: 0.8653

Epoch 00003: val_loss improved from 0.59847 to 0.57408, saving model to /content/drive/MyDrive/Dockship_D

In [27]:
saved_model = tf.keras.models.load_model('/content/drive/MyDrive/Dockship_Dataset/Models/best_model.hdf')

In [28]:
#function which will return the probabilities of respective classes and the type of object
def predict(path):
  img = image.load_img(path=path ,target_size=(200, 200,3))
  img_array = image.img_to_array(img)
  img_batch = np.expand_dims(img_array, axis=0)
  # print(model.predict_classes(img_batch))
  prediction = saved_model.predict(img_batch)
  result= np.argmax(prediction,axis=1)
  class_indices = list(valid_generator.class_indices.keys())
  return (class_indices[result[0]],max(prediction[0])*100)

In [29]:
predict('/content/drive/MyDrive/Dockship_Dataset/Garbage_Classification_AI_Challenge/garbage_classification_ai_challenge-dataset/TEST/001.jpg')

('metal', 99.83800649642944)

In [30]:
result = []
for add in test_input:
  result.append(predict(add)[0])

In [31]:
result[:10]

['metal',
 'trash',
 'glass',
 'metal',
 'plastic',
 'glass',
 'metal',
 'metal',
 'paper',
 'glass']

In [32]:
list(test_output[:10])

['metal',
 'trash',
 'glass',
 'metal',
 'plastic',
 'glass',
 'metal',
 'metal',
 'paper',
 'glass']

In [33]:
confusion_matrix(list(test_output),result)

array([[65,  0,  0,  0,  0,  0],
       [ 0, 75,  4,  0,  1,  0],
       [ 0,  0, 66,  0,  0,  0],
       [ 0,  1,  7, 86,  0,  1],
       [ 0,  1,  2,  0, 72,  2],
       [ 0,  0,  0,  3,  0, 15]])

In [34]:
history.history.keys()

NameError: ignored

In [ ]:
plt.plot(range(1,9),history.history['val_accuracy'],color='red',label='val_accuracy')
plt.plot(range(1, 9),history.history['accuracy'],color='blue',label='train_accuracy')
plt.legend()

In [ ]:
history.history['loss']

In [ ]:
plt.plot(range(1,9),history.history['loss'],color='blue',label='loss')
plt.plot(range(1,9),history.history['val_loss'],color='red',label='val_loss')
plt.legend()

In [35]:
dir = '/content/drive/MyDrive/Dockship_Dataset/Garbage_Classification_AI_Challenge/garbage_classification_ai_challenge-dataset/TEST'
images = os.listdir(dir)

In [64]:
test_result = []
for filename in images:
  img = image.load_img(path="/content/drive/MyDrive/Dockship_Dataset/Garbage_Classification_AI_Challenge/garbage_classification_ai_challenge-dataset/TEST/"+str(filename) ,target_size=(200, 200,3))
  img_array = image.img_to_array(img)
  img_batch = np.expand_dims(img_array, axis=0)
  pred = saved_model.predict(img_batch)
  prediction= np.argmax(pred,axis=1)
  test_result.append(prediction)

In [65]:
pred = np.array(test_result).ravel()
labels = dict((v,k) for k,v in (valid_generator.class_indices).items())
pred = [labels[k] for k in pred]

In [66]:
df_new = pd.DataFrame(list(zip(images, pred)),columns = ['Filename','Labels'])

In [67]:
df_new

,Filename,Labels
0,251.jpg,plastic
1,081.jpg,metal
2,113.jpg,paper
3,142.jpg,glass
4,254.jpg,metal
...,...,...
295,212.jpg,paper
296,008.jpg,glass
297,250.jpg,cardboard
298,189.jpg,plastic


In [68]:
df_new.to_csv("/content/drive/MyDrive/Dockship_Dataset/Submissions/Garbage_Classification_submission.csv", index = False)